In [1]:
%pip install langchain
%pip install faiss-cpu
%pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pickle
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import numpy as np

/Users/vidbebek/Library/Python/3.9/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/vidbebek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class DataHandler:
    def __init__(self):
        pass
    
    def ucitaj_dataset(self, directory):
        text_data = []
        for filename in os.listdir(directory):
            if filename.endswith(".txt"):
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r', encoding='ISO-8859-1') as file:
                    text_data.append(file.read())
        return text_data

    def podijeli_na_chunkove(self, text_data, text_splitter):
        chunks = []
        for text in text_data:
            chunks.extend(text_splitter.split_text(text))
        
        return chunks
    
    def zapisi_chunkove(self, chunks):
        with open('chunks.pkl', 'wb') as f:
            pickle.dump(chunks, f)

In [4]:
dataHandler = DataHandler()
text_data = dataHandler.ucitaj_dataset('dataset')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10, length_function=len)
chunks = dataHandler.podijeli_na_chunkove(text_data, text_splitter)

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
chunk_vectors = model.encode(chunks)

dim = chunk_vectors.shape[1]
index = faiss.IndexFlatL2(dim)

index.add(np.array(chunk_vectors))

faiss.write_index(index, "faiss_index.index")

dataHandler.zapisi_chunkove(chunks)

print('Zavrseno kreiranje baze')

Zavrseno kreiranje baze
